In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup
import re
import pandas as pd

In [6]:
ü = "%C3%BC"
ß = "%C3%9F"
#street = None

In [7]:
luenen_files = pd.read_csv("Luenen.csv", delimiter=";", encoding="latin-1", names=["Street", "District", "Latitude", "Longitude"])
street_list = luenen_files["Street"]

In [8]:
district_street = [luenen_files["District"], luenen_files["Street"]]

In [9]:
district_street[1]

0               Achenbachstraße
1                   Ackerstraße
2                   Ackerstraße
3          Adalbert-Stifter-Weg
4        Adolf-Damaschke-Straße
5            Adolf-Stock-Straße
6                   Ahornstraße
7                      Ährenweg
8                 Akazienstraße
9      Albert-Schweitzer-Straße
10       Alfred-Meermann-Straße
11          Alfred-Seepe-Straße
12                  Alfredplatz
13          Allensteiner Straße
14       Aloys-Siegeroth-Straße
15                  Alsenstraße
16             Alstedder Straße
17                 Altdorferweg
18               Alter Kirchweg
19                Alter Postweg
20               Alte Gärtnerei
21              Alte Herrenthey
22                Alte Ziegelei
23               Altstadtstraße
24            Am Alten Sägewerk
25                     Am Anger
26                 Am Brambusch
27               Am Calversbach
28             Am Christinentor
29                      Am Diek
                 ...           
634     

In [10]:
luenen_files["District"].unique()

array(['Brambauer', 'Nordlünen', 'Alstedde', 'Lünen-Süd', 'Niederaden',
       'Wethmar', 'Osterfeld', 'Horstmar', 'Lünen-Nord', 'Beckinghausen',
       'Lünen', 'Lippholthausen', 'Gahmen', 'Horstmar/Lünen-Süd',
       'Geistviertel', 'Mitte', 'Lünen-Nord, gleich hinter der Lippe'],
      dtype=object)

In [12]:
#notwendig?
tmp_url = "https://www.dasoertliche.de/?form_name=search_nat_ext&ci=L%C3%BCnen-S%C3%BCd+-+Stadt+L%C3%BCnen&context=11&page=78&action=56&st=Adolf-Damaschke-Stra%C3%9Fe&zvo_ok=0&kgs=05978024000&radius=0&orderby=name&ttforderby=rel&recFrom=1"
response = get(tmp_url)

#print(response.text[:500])

<!DOCTYPE html>
<html lang="de">
<head>
	<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width, user-scalable=yes">
<link rel="canonical" href="https://www.dasoertliche.de/Themen/Adolf--Damaschke--Straße/Lünen--Süd----Stadt-Lünen.html">
<link rel="alternate" media="only screen and (max-width: 640px)" href="https://mobil.dasoertliche.de/Themen/Adolf--Damaschke--Straße/Lünen--Süd----Stadt-Lünen.html">
<link rel="alternate" href="android-app://de.dasoertliche.android/https/mo


In [11]:
url = "https://www.dasoertliche.de/?form_name=search_nat_ext&ci=L{}nen-S{}d+-+Stadt+L{}nen&context=11&page=78&action=56&st={}&zvo_ok=0&kgs=05978024000&radius=0&orderby=name&ttforderby=rel&recFrom=1"
urls = []
district = []
for street in range(len(district_street[1])):
    streets = district_street[1][street]
    streets = re.sub("\s+", "+", streets.strip())    
    urls.append(url.format(ü,ü,ü,streets))
    district.append(district_street[0][street])
    #print(url.format(ü,ü,ü,street), "\n\n")

In [162]:
def get_container(tmp_url):
    response = get(tmp_url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    address_containers = html_soup.find_all('div', class_='hit clearfix')
    results = (len(address_containers))
    return address_containers


def get_content(address_containers):
    name = []
    street = []
    for i in range(len(address_containers)):
        name.append(address_containers[i].span.text)
        street.append(address_containers[i].address.text)
    return name, street

In [164]:
# Iterate over list of all urls
all_districts = {}
for url in range(len(urls)):
    print(url)

    page_two = []
    log = []
    #print(urls[url])
    # call Function
    response = get(urls[url])
    html_soup = BeautifulSoup(response.text, 'html.parser')
    paging_container = html_soup.find_all('div', class_='paging')
    address_containers = get_container(urls[url])

    sites = {}
    tmp_counter = 1
    try:
        get_all_a = paging_container[0].find_all('a', href=True)
        for a in get_all_a:
            if tmp_counter == 1:
                name, street = get_content(address_containers)
                all_districts[district[url]] = [name, street]
                #print(test)
            if len(get_all_a) != tmp_counter and tmp_counter > 1:
                #print("mehr als eine Seite abarbeiten!!!")
                for i in range(1, len(paging_container[0].find_all('a'))-1):
                    other_page_url = paging_container[0].find_all('a')[i].get('href')
                    address_containers = get_container(a['href'])
                    name, street = get_content(address_containers)
                    all_districts[district[url]] = [name, street]

            tmp_counter += 1
    except:
        log.append(urls[url])
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

ConnectionError: HTTPSConnectionPool(host='www.dasoertliche.de', port=443): Max retries exceeded with url: /?form_name=search_nat_ext&ci=L%C3%BCnen-S%C3%BCd+-+Stadt+L%C3%BCnen&context=11&page=78&action=56&st=Wagnerstra%C3%9Fe&zvo_ok=0&kgs=05978024000&radius=0&orderby=name&ttforderby=rel&recFrom=1 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000024A691D6278>: Failed to establish a new connection: [WinError 10060] Ein Verbindungsversuch ist fehlgeschlagen, da die Gegenstelle nach einer bestimmten Zeitspanne nicht richtig reagiert hat, oder die hergestellte Verbindung war fehlerhaft, da der verbundene Host nicht reagiert hat'))

    name = []
    street = []
    page_two = []
    log = []

In [158]:
all_districts

{}

In [148]:
cumm = []
for i in all_districts["Brambauer"][0]:
    
    cumm.append(i)

In [149]:
len(cumm)

14

# Test area for multipage
tmp_multi_url = "https://www.dasoertliche.de/?form_name=search_nat_ext&ci=L%C3%BCnen-S%C3%BCd+-+Stadt+L%C3%BCnen&context=11&page=78&action=56&st=Am+Katzbach&zvo_ok=0&kgs=05978024000&radius=0&orderby=name&ttforderby=rel&recFrom=1"

response = get(tmp_multi_url)
html_soup = BeautifulSoup(response.text, 'html.parser')

paging_container = html_soup.find_all('div', class_='paging')

sites = {}
tmp_counter = 1
get_all_a = paging_container[0].find_all('a', href=True)
for a in get_all_a:
    print(tmp_counter)
    if len(get_all_a) != tmp_counter and tmp_counter > 1:
        sites[tmp_counter] = a['href']
    tmp_counter += 1

In [17]:
address_containers = get_container(tmp_multi_url)
len(address_containers)

24

In [184]:
all_districts

{'Brambauer': [['Akcin Bayram\xa0',
   'Aygül Pala\xa0',
   'Eroglu Nurettin\xa0',
   'Exner Margith\xa0',
   'Göhlich Anni\xa0',
   'Ismailat Abboud\xa0',
   'Maas Susanne u. Hans-Joachim\xa0',
   'Manahl Michael\xa0',
   'Mutlu Kenan\xa0',
   'Neb Alexander\xa0',
   'Okan Andrea\xa0',
   'Sengül Osman\xa0',
   'Serir Krystyna\xa0',
   'Sivathasan Kanthiah\xa0',
   'Akcin Bayram\xa0',
   'Aygül Pala\xa0',
   'Eroglu Nurettin\xa0',
   'Exner Margith\xa0',
   'Göhlich Anni\xa0',
   'Ismailat Abboud\xa0',
   'Maas Susanne u. Hans-Joachim\xa0',
   'Manahl Michael\xa0',
   'Mutlu Kenan\xa0',
   'Neb Alexander\xa0',
   'Okan Andrea\xa0',
   'Sengül Osman\xa0',
   'Serir Krystyna\xa0',
   'Sivathasan Kanthiah\xa0',
   'Akcin Bayram\xa0',
   'Aygül Pala\xa0',
   'Eroglu Nurettin\xa0',
   'Exner Margith\xa0',
   'Göhlich Anni\xa0',
   'Ismailat Abboud\xa0',
   'Maas Susanne u. Hans-Joachim\xa0',
   'Manahl Michael\xa0',
   'Mutlu Kenan\xa0',
   'Neb Alexander\xa0',
   'Okan Andrea\xa0',
   'Se

In [185]:
len(all_districts["Lünen-Süd"][0])

150

In [156]:
#links = bsObj.findAll('a', href=re.compile('(cymbeline)'))
pages = paging_container[0].find_all('a')
for i in range(1, len(paging_container[0].find_all('a'))-1):
    print(paging_container[0].find_all('a')[i])

<a href="https://www.dasoertliche.de/?wntHit=&amp;kw=&amp;form_name=search_nat_ext&amp;ci=L%c3%bcnen-S%c3%bcd+-+Stadt+L%c3%bcnen&amp;context=11&amp;page=78&amp;action=56&amp;st=Adolf-Damaschke-Stra%c3%9fe&amp;zvo_ok=0&amp;kgs=05978024000&amp;radius=0&amp;orderby=name&amp;ttforderby=rel&amp;recFrom=26&amp;orderby=name&amp;ttforderby=rel" rel="nofollow" title="zur Seite 2">2</a>


In [83]:
address_containers[0].find('div', class_ = 'left oe_hit')


<div class="left oe_hit">
<div class="counter">1</div>
<h2><a class="name" href="https://www.dasoertliche.de/?id=0465000160140425676084&amp;recuid=7HUFDXJNEKVJRGUNMRHEKWBM6RLHE2CKL74GIWS2GNZMGAQ&amp;action=58&amp;pagePos=1&amp;dar=1&amp;kw=&amp;form_name=detail&amp;lastFormName=search_nat_ext&amp;ci=L%c3%bcnen-S%c3%bcd+-+Stadt+L%c3%bcnen&amp;recFrom=1&amp;st=Adolf-Damaschke-Stra%c3%9fe&amp;hitno=0&amp;kgs=05978024000&amp;zvo_ok=0&amp;radius=0&amp;orderby=name&amp;ttforderby=rel&amp;buc=465&amp;showbuc=0&amp;verlNr=166&amp;page=78&amp;context=11" onmouseout="" onmouseover=""><span class="">Bartsch Andrea Fachärztin für Allgemeinmedizin ,  Stephan Dr. Facharzt für Innere Medizin </span>
</a></h2>
<address>
					Adolf-Damaschke-Str. 2, 44532 <span class="">Lünen, Süd</span><span id="itranking_0"></span>
</address>
<a class="hitlnk_mail_incog" href="#" title="zur Detailansicht">E-Mail</a>
<a class="hitlnk_url_incog" href="#" title="zur Detailansicht">Web<span class="st-web-url gone">www.ba

In [149]:
# Telefonnummer muss auf die Seite gedrückt werden um es einzeln aufzurufen
address_containers[0].find("a", "mask-nr").span.text

'(0 23 06) ...'

In [151]:
for i in range(len(address_containers)):
    print( address_containers[i].span.text,address_containers[i].address.text)
 
    

Bartsch Andrea Fachärztin für Allgemeinmedizin ,  Stephan Dr. Facharzt für Innere Medizin  
					Adolf-Damaschke-Str. 2, 44532 Lünen, Süd

Berger Wolfgang  
					Adolf-Damaschke-Str. 3, 44532 Lünen, Süd

Bluck Manfred  
					Adolf-Damaschke-Str. 17, 44532 Lünen, Süd

Dr. med. Andreas Michael Zabel u. Dr. med. K. P. Zimmermann  
					Adolf-Damaschke-Str. 2, 44532 Lünen, Süd

Ferlmann Carsten Zahnarztpraxis  
					Adolf-Damaschke-Str. 2, 44532 Lünen, Süd

Gillmeister Michael  
					Adolf-Damaschke-Str. 33, 44532 Lünen, Süd

Gillmeister Sigrid  
					Adolf-Damaschke-Str. 33, 44532 Lünen, Süd

Herrmann Arno  
					Adolf-Damaschke-Str. 25, 44532 Lünen, Süd

Hinz Helmut  
					Adolf-Damaschke-Str. 11, 44532 Lünen, Süd

Hoppe Christa  
					Adolf-Damaschke-Str. 21, 44532 Lünen, Süd

Keiser Herbert  
					Adolf-Damaschke-Str. 17, 44532 Lünen, Süd

Kobialka Hannelore  
					Adolf-Damaschke-Str. 23, 44532 Lünen, Süd

Kurt Franziska  
					Adolf-Damaschke-Str. 5, 44532 Lünen, Süd

Mähner Nicole D